<a href="https://colab.research.google.com/github/Celestial00/DS-A/blob/main/20sw050(lab9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import sqlite3
import pandas as pd
import numpy as np

# Generator for reading in one line at a time
def extract_lines(file):
    while True:
        line = file.readline()
        if not line:
            break
        yield line
# Transform the indicator data
# Transform the indicator data
# Transform the indicator data
def transform_indicator_data(data, colnames):
    # Get rid of quote marks
    for i, datum in enumerate(data):
        data[i] = datum.replace('"', '')

    # Check if the line represents column names (header row)
    if data[0] == 'Country Name':
        colnames = data
    elif data[0] != '﻿Data Source' and len(data) >= len(colnames):  # Check if the line has enough elements
        # Extract the countryname and countrycode from the list
        country = data[0]
        countrycode = data[1]

        # List of non-country values to filter out
        non_countries = ['World', 'High income', 'OECD members', 'Post-demographic dividend', 'IDA & IBRD total', 'Low & middle income', 'Middle income', 'IBRD only', 'East Asia & Pacific', 'Europe & Central Asia', 'North America', 'Upper middle income', 'Late-demographic dividend', 'European Union', 'East Asia & Pacific (excluding high income)', 'East Asia & Pacific (IDA & IBRD countries)', 'Euro area', 'Early-demographic dividend', 'Lower middle income', 'Latin America & Caribbean', 'Latin America & the Caribbean (IDA & IBRD countries)', 'Latin America & Caribbean (excluding high income)', 'Europe & Central Asia (IDA & IBRD countries)', 'Middle East & North Africa', 'Europe & Central Asia (excluding high income)', 'South Asia (IDA & IBRD)', 'South Asia', 'Arab World', 'IDA total', 'Sub-Saharan Africa', 'Sub-Saharan Africa (IDA & IBRD countries)', 'Sub-Saharan Africa (excluding high income)', 'Middle East & North Africa (excluding high income)', 'Middle East & North Africa (IDA & IBRD countries)', 'Central Europe and the Baltics', 'Pre-demographic dividend', 'IDA only', 'Least developed countries: UN classification', 'IDA blend', 'Fragile and conflict affected situations', 'Heavily indebted poor countries (HIPC)', 'Low income', 'Small states', 'Other small states', 'Not classified', 'Caribbean small states', 'Pacific island small states']

        # Filter out country name values that are in the above list
        if country not in non_countries:
            # Initialize an empty results list
            results = []

            # Iterate through the rows in df_melt
            for index, year_value in enumerate(colnames[2:]):  # Skip the first two columns (Country Name and Country Code)
                if year_value.strip() != '' and index < len(data) - 2:  # Check if the year_value is not empty and the index is within bounds
                    try:
                        year = int(year_value)
                        gdp = data[2 + index]  # The GDP value at the corresponding index

                        # Check if gdp can be converted to a float and is not NaN
                        if gdp.strip() != '':
                            gdp = float(gdp)
                            results.append([country, countrycode, year, gdp])
                    except (ValueError, TypeError) as e:
                        print(f"Error processing year_value: '{year_value}' for country: '{country}'")
                        print(e)

            return results


# Load indicator data into the SQLite database
def load_indicator_data(results):
    # Connect to the worldbank.db database using the sqlite3 library
    conn = sqlite3.connect('worldbank.db')

    # Create a cursor object
    cur = conn.cursor()

    if results:
        # Iterate through the results variable and insert each result into the gdp table
        for result in results:
            countryname, countrycode, year, gdp = result

            # Prepare a query to insert a countryname, countrycode, year, gdp value
            sql_string = "INSERT INTO gdp (countryname, countrycode, year, gdp) VALUES (?, ?, ?, ?);"

            # Connect to the database and execute the query
            try:
                cur.execute(sql_string, (countryname, countrycode, year, gdp))
            # Print out any errors (like if the primary key constraint is violated)
            except Exception as e:
                print('Error occurred:', e, result)

    # Commit changes and close the connection
    conn.commit()
    conn.close()

# Create the database and table
def create_database_and_table():
    # Connect to the database
    conn = sqlite3.connect('worldbank.db')

    # Get a cursor
    cur = conn.cursor()

    # Drop the table if it already exists
    cur.execute("DROP TABLE IF EXISTS gdp")

    # Create the gdp table including countrycode and year as primary keys
    cur.execute("CREATE TABLE gdp (countryname TEXT, countrycode TEXT, year INTEGER, gdp REAL, PRIMARY KEY (countrycode, year));")

    conn.commit()
    conn.close()

# Execute this code cell to run the ETL pipeline
# Open the data file
with open('gdp_data.csv') as f:
    # Execute the generator to read in the file line by line
    for line in extract_lines(f):
        # Split the comma-separated values
        data = line.split(',')
        # Transform and load the line of indicator data
        results = transform_indicator_data(data, colnames)
        if results:
            load_indicator_data(results)

# Create the database and table
create_database_and_table()

# Display the resulting DataFrame
conn = sqlite3.connect('worldbank.db')
df = pd.read_sql("SELECT * FROM gdp", con=conn)
conn.close()
df


Error processing year_value: 'Indicator Name' for country: 'Aruba'
invalid literal for int() with base 10: 'Indicator Name'
Error processing year_value: 'Indicator Code' for country: 'Aruba'
invalid literal for int() with base 10: 'Indicator Code'
Error processing year_value: 'Indicator Name' for country: 'Afghanistan'
invalid literal for int() with base 10: 'Indicator Name'
Error processing year_value: 'Indicator Code' for country: 'Afghanistan'
invalid literal for int() with base 10: 'Indicator Code'
Error processing year_value: 'Indicator Name' for country: 'Angola'
invalid literal for int() with base 10: 'Indicator Name'
Error processing year_value: 'Indicator Code' for country: 'Angola'
invalid literal for int() with base 10: 'Indicator Code'
Error processing year_value: 'Indicator Name' for country: 'Albania'
invalid literal for int() with base 10: 'Indicator Name'
Error processing year_value: 'Indicator Code' for country: 'Albania'
invalid literal for int() with base 10: 'Indica

,countryname,countrycode,year,gdp
